In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = (
    SparkSession.builder.appName("preprocessing of taxi data")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/15 01:35:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### **PlanA**: Cluster merchants
#### Attributes: average dollar amount, number of unique customers, total number of orders

#### **PlanB**: Classify merchants
#### Attributes: order history of customer who purchase product in the missing merchant, average dollar amount, number of unique customers, total number of orders
### **Classification pipeline**:
 1. Data Engineering
  * Prepare data into ready-to-use format
    * Create a new dataframe from the original dataframe which relates unknown merchants to order history of its customers who purchases from at least one known merchant stores.
    * Clean the prod_desc
    * Create a new dataframe which retains only the useful attributes for revenue level and take rate
  * Need one curated dataset for modeling product description and one dataset for modeling revenue level and take rate
 2. Feature Engineering
  * Aggregate data to produce more useful features for modeling revenue level and take rate
  * Recommended features for prod_desc: dollar value, consumer id/ user id and consumer gender
  * Recommended features for revenue level and take rate: average dollar amount per month/week/day, total number of orders per month/week/day and number of distinct customers per month/week/day
 3. Data Modeling
  * Discuss and finalize choice of model: XGBoost, MLP, NB(last resort)
  * Fitting and Tuning model to achieve optimal performance 
 4. Model Validation
 * Metrics:
    * Categorical(prod_desc and revenue_level): 
      * Accuracy
      * f1 score
    * Continuous(take_rate):
      * RMSE
 * Visualization:
    * Categorical:
      * learning curve
      * ROC curve
      * confusion matrix 
    * Continuous:
      * RMSE vs. fitted value
 5. Model deployment
   * Use the prediction to impute missing information

In [28]:
transaction_20210828_20220227_sdf = spark.read.parquet("../data/curated/transactions_20210828_20220227_all_detail")
transaction_20210828_20220227_missings_sdf = spark.read.parquet("../data/curated/transactions_20210828_20220227_all_detail_missing_counts")
transaction_20210228_20210827_sdf = spark.read.parquet("../data/curated/transactions_20210228_20210827_all_detail")

In [5]:
num_of_unknown_merchants = transaction_20210828_20220227_sdf.where(F.col("merchant_name").isNull()) \
                            .select(F.col("merchant_abn")).distinct().count()
num_of_order_from_unknown_merchants = transaction_20210828_20220227_sdf.where(F.col("merchant_name").isNull()) \
                            .select(F.col("merchant_abn")).count()

print(f"num_of_unknown_merchants = {num_of_unknown_merchants}" + "\n" +
      f"num_of_order_from_unknown_merchants = {num_of_order_from_unknown_merchants}")

num_of_unknown_merchants = 381
num_of_order_from_unknown_merchants = 184414


### Checking if every missing merchant has at least one related known merchant, i.e. if there is at least one customer who buys from an unknown merchant also buy from other merchants

In [6]:
transaction_20210828_20220227_sdf.createOrReplaceTempView('transactions')

spark.sql("""
SELECT merchant_abn, user_id
FROM transactions
WHERE merchant_name IS NULL
""").createOrReplaceTempView('unknown_merchants')

spark.sql("""
SELECT *
FROM transactions
WHERE merchant_name IS NOT NULL
""").createOrReplaceTempView('orders_in_known_merchants')


joined_sdf = spark.sql("""
SELECT table1.merchant_abn AS unknown_merchant_abn, 
    COUNT(table1.merchant_abn) AS num_of_order_in_known_merchant,
    COUNT(DISTINCT table2.user_id) AS num_of_distinct_customers,
    COUNT(DISTINCT table2.merchant_abn) AS num_of_known_merchant,
    COUNT(DISTINCT table2.prod_desc) AS num_of_distinct_prod_desc,
    COUNT(DISTINCT table2.revenue_level) AS num_of_distinct_revenue_level
FROM unknown_merchants AS table1
LEFT JOIN orders_in_known_merchants AS table2 
ON table1.user_id=table2.user_id
GROUP BY table1.merchant_abn
""")

joined_sdf.limit(5)

unknown_merchant_abn,num_of_order_in_known_merchant,num_of_distinct_customers,num_of_known_merchant,num_of_distinct_prod_desc,num_of_distinct_revenue_level
24406529929,235349,1276,3394,116,5
56395390867,3249,18,992,67,5
28767881738,348,2,242,32,5
98345415950,735,4,413,41,5
45925655949,6196,35,1361,82,5


In [7]:
joined_sdf.filter(F.col("num_of_order_in_known_merchant") == 0)

unknown_merchant_abn,num_of_order_in_known_merchant,num_of_distinct_customers,num_of_known_merchant,num_of_distinct_prod_desc,num_of_distinct_revenue_level


In [8]:
# Need to clean the data
transaction_20210828_20220227_sdf.select(F.col("prod_desc")).distinct().orderBy(F.col("prod_desc")).take(5)

[Row(prod_desc=None),
 Row(prod_desc='antique shops -  sales, repairs, and restoration services'),
 Row(prod_desc='antique shops - sales,  repairs, and restoration services'),
 Row(prod_desc='antique shops - sales, repairs, and restoration services'),
 Row(prod_desc='art dealers and  galleries')]

In [27]:
transaction_20210828_20220227_sdf.select(F.col("prod_desc")).distinct().orderBy(F.col("prod_desc")).count()



[Row(prod_desc='florists supplies, nursery stock, and flowers'),
 Row(prod_desc='furniture, home furnishings and equipment shops, and manufacturers, except appliances'),
 Row(prod_desc='gift, card, novelty, and souvenir shops'),
 Row(prod_desc='health and beauty spas'),
 Row(prod_desc='hobby, toy and game shops'),
 Row(prod_desc='jewelry, watch, clock, and silverware shops'),
 Row(prod_desc='lawn and garden supply outlets, including nurseries'),
 Row(prod_desc='motor vehicle supplies and new parts'),
 Row(prod_desc='music shops - musical instruments, pianos, and sheet music'),
 Row(prod_desc='opticians, optical goods, and eyeglasses'),
 Row(prod_desc='shoe shops'),
 Row(prod_desc='stationery, office supplies and printing and writing paper'),
 Row(prod_desc='telecom'),
 Row(prod_desc='tent and awning shops'),
 Row(prod_desc='watch, clock, and jewelry repair shops')]

In [9]:
# take_rate is the independent of the dollar amount of an order
transaction_20210828_20220227_sdf.orderBy(F.col("merchant_abn")).limit(5)

user_id,merchant_abn,dollar_value,order_id,order_datetime,merchant_name,prod_desc,revenue_level,take_rate,consumer_name,address,state,postcode,gender,consumer_id
1277,10023283211,313.00851867046595,654b3e5c-60cf-486...,2021-11-28,Felis Limited,"furniture, home f...",e,0.1,Tara Howard,904 Earl Light,QLD,4877,Female,1104483
2700,10023283211,182.36576363575438,347d05e2-202d-412...,2021-11-02,Felis Limited,"furniture, home f...",e,0.1,Danielle Wang,6442 Lee Ridge,VIC,8002,Female,1331269
1374,10023283211,192.56967025206546,bc18bc09-926c-444...,2022-02-27,Felis Limited,"furniture, home f...",e,0.1,Mark Garcia,9450 Martinez Knolls,QLD,4116,Male,561397
202,10023283211,284.28664583466747,5c8fbf60-4dfe-416...,2021-12-05,Felis Limited,"furniture, home f...",e,0.1,Bryan Ho,8010 Denise Gardens,NSW,1485,Male,953975
1640,10023283211,367.06151947286946,24bebd74-9f0c-440...,2021-11-05,Felis Limited,"furniture, home f...",e,0.1,Jenna Page,1882 Courtney Tunnel,QLD,4655,Female,1036773


In [10]:
# Take rate and revenue level are highly negatively correlated
from pyspark.ml.stat import Correlation
from pyspark.sql.types import ByteType, FloatType
revenue_dict = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5}

@F.udf(returnType=ByteType())
def revenue_level_converter(rl):
    return revenue_dict[rl]

transaction_20210828_20220227_sdf.filter(F.col("revenue_level").isNotNull()) \
                                 .withColumn("take_rate", F.col("take_rate").cast(FloatType())) \
                                 .withColumn("revenue_level_int", revenue_level_converter(F.col("revenue_level"))) \
                                 .corr('revenue_level_int', 'take_rate')

-0.9520944324849947